# SDC Polyglot Workbook 

This work book is an example of what can be done with SDC in Polyglot notebook. 

In [ ]:
 <Section name="S_4257" order="19" ID="4257.100004300" minCard="0">
 </Section>

## Sections
 
The code snippet above shows a section with a name, order, ID and minCard attribute

# Questions

Each QAS in the FDF XML is an XFC that can be represented in the DEF by one or more controls (widgets) by the FF software. The controls capture user response data in the DEF.  Each QAS XFC has an ID   and other metadata. DEF controls are assigned the matching XFC ID to enable data from the DEF control to be captured into the XFC XML that shares its @ID. The behavior of the control is determined by the corresponding XFC metadata. There are 3 QAS types: Question-Response, Single-Select Question, and Multi-Select Question.

The primary visible text for all controls is found in the title attribute inside the `<Question>` element.  

## Question-Response

The QR control represents a Question control with an adjacent area for capturing a response from the user. The QR XFC does not have an answer list (a List of ListItems). 

A QR response may be validated by rules in the Response element, which determine if the inputs are in the proper format (e.g., as a text string or decimal, or rarely, following a pattern mask or a binary type).

**Observe the following points about this sample:**

* *Question* is the only XFC that appears in this XML example. The other nested elements support the *Question XFC* with additional metadata.

* All XFC elements (*Question* in this case) must have an **ID** that is unique within the FDF. This uniqueness is enforced by the XML Schema. Uniqueness of IDs must be confirmed by SDC-compliant software. Since XFCs can be repeated in DEF implementation, uniqueness enforcement is an important task in SDC software implementations.

* The text to display in the DEF is in the *title content*.

* **Responses** are captured in a `ResponseField / Response / (datatype)` structure, where `(datatype)` is an SDC element corresponding to one of the SDC datatypes.

  * Responses must contain an SDC datatype element (*string* in this case).

  * User-entered responses are captured in the `val` attribute of the `(datatype)` element, like this:
    ```xml
    <string maxLength="4000" minLength="0" val="This is my comment" />
    ```

    The example below offers a complete example of a `<Question>` element. 

In [ ]:
<Question ID="40273.100004300" title="Comment(s)">
    <ResponseField>
        <Response>
            <string maxLength="4000" minLength="0"/>
        </Response>
    </ResponseField>
</Question>

### Repsponse Field

As mentioned in the Question-Response section, the response field is designed to take user input of some kind with the as specified datatypes. SDC supports W3C datatypes and also supports XML and HTML datatypes for user responses. Example datatypes can include strings, integers, decimals, and dates.

* All `datatype` elements names use lower camel case, unlike all other elements in the SDC Schema. This is done to conform with the W3C representation of the datatypes in XML Schema documents. One notable exception because of it's importance is `@ID`

* All datatypes are defined and modeled after the W3C datatypes and include all of the W3C datatype metadata attributes.

* All metadata attributes have a type appropriate for the element and the metadata. For example, `maxLength` has an XML Schema datatype of `xs:long`.

* The `val` attribute validates the datatype of its element tag. Entering an invalid datatype in this attribute will generate a validation error when validated with the SDCFormDesign Schema.

* `val` cannot be Schema-validated against the element's other datatype metadata (for `string`, in this case, `maxLength`, `minLength`, `mask`, `pattern`), so that further validation of `val` content must use additional methods such as Schematron.

Here is another example that shows how numeric QR structures may appear with:


In [ ]:
<Question ID="QR1" title="Specify Percentage of QR quarks (see Note A)">
    <ResponseField>
        <Response>
            <integer maxInclusive="100" minInclusive="1"/>
        </Response>
        <TextAfterResponse val="%"/>
        <ResponseUnits val="%" unitSystem="UCUM"/>
    </ResponseField>
</Question>

* The example above uses an `<integer>` datatype to specify and record the user-entered value.
* When the user enters a response into the DEF, the `<integer>` element will be populated with `val` and look like this:

```xml
  <integer maxInclusive="100" minInclusive="1" val="My user response"/>
```

* The displayed datatype validation metadata are specific for the `integer` datatype (`maxInclusive`, `minInclusive`).
* Other attributes are available for numeric response datatypes, including: `quantEnum`, `maxExclusive`, `minExclusive`, `totalDigits`, `mask`, `allowGT`, `allowGTE`, `allowLT`, `allowLTE`, and `allowAPPROX`. For decimals and float, `fractionDigits` is also available.
* The `TextAfterResponse` element provides a place to specify static text that should appear after the user’s response in the DEF and the report. The `val` content holds the text for display.
* The `ResponseUnits` element provides a place to record the units relevant to the response. The `val` attribute holds the text ("%") of the units for the user’s response. This is provided by the FDF designer, not the DEF-user. The `ResponseUnits` is “hard-coded” into the FDF and is not changeable via the DEF. The `unitSystem` attribute defaults to “UCUM” (Unified Code for Units of Measure) and is not usually displayed in the DEF or report.

## Single Select Questions

A QS control displays a Question that subsumes a List of ListItem controls.  The List is set to single-select in the XFC metadata and in the DEF control.  The QS control is often rendered as a Question title area along with option/radio buttons for the LIs, or as a combo-box with dropdown answer choices, from which only one answer may be selected.  In the XFC, the QS ListField element has maxSelections="1" which is the default value for this attribute; if maxSelections does not appear in the ListField (as in the example below), then the Question is treated as a QS.


In [ ]:
<Question ID="QS1" title="Select your favorite color for QS">
    <ListField>
    <!-- ListField maxSelections="1" is not shown since it is the default value-->
        <List>
            <ListItem ID="LI1" title="Baseball"/>
            <ListItem ID="LI2" title="Zebra"/>
            <ListItem ID="LI3" title="Snort" selected="true"/>
            <ListItem ID="LI4" title="Zinger"/>
            <ListItem ID="LI5" title="Enter another color" >
        </List>
    </ListField>
</Question>


### ListItemResponseField

The `<ListItemResponseField>` is used when a list requires an "Other (Specify):" style question that requires user input outside of the items displayed in the list. It can contain the same datatypes as the `<ResponseField>` element. Both contain the `<Response>` element which contains the element that defines the datatype for answer (e.g. `<string>`).

In [ ]:
<Question ID="QS1" title="Select your favorite color for QS">
    <ListField> 
        <List>
            <ListItem ID="LI1" title="Baseball"/>
            <ListItem ID="LI2" title="Zebra"/>
            <ListItem ID="LI3" title="Snort" selected="true"/>
            <ListItem ID="LI4" title="Zinger"/>
            <ListItem ID="LI5" title="Enter another color" >
                <Property propName="reportText" val="{no text}"/>
                <ListItemResponseField responseRequired="true">
                    <Response>
                        <string maxLength="30" />
                    </Response>
                </ListItemResponseField>
            </ListItem>
        </List>
    </ListField>
</Question>


In this XML snippet, the ListItemResponseField element is associated with the ListItem labeled "Enter another color". This specific ListItemResponseField serves the purpose of capturing user responses when the user selects this particular option when the listed options cannot be used. 

## Multi-select Questions



In [ ]:
<Question ID="QM1" title="This is a QM Question title" >
    <ListField maxSelections="0">
        <List>
            <ListItem ID="LI1" title="ListItem.title.1" />
            <ListItem ID="LI2" title="ListItem.title.2" />
            <ListItem ID="LI3" title="ListItem.title.3" />
        </List>
    </ListField>
</Question>

## minCard

@minCard is the minimum number of repetitions allowed for a section or question. The default value is 1, indicating that the user must answer any question(s) that has this attribute set. If @minCard is set to 0, then the item and all descendent questions are optional to answer.


In [ ]:
<Question name="Q_2118" order="21" ID="2118.100004300" minCard="0" readOnly="true">
      <Property name="p_altTxt_2118_1" order="22" propName="altText" val="Tumor Site" />
      <ListField name="lf_2118_2" order="23">
       <List name="lst_2118_3" order="24">
        <ListItem name="LI_2119" order="25" ID="2119.100004300" title="Adrenal gland" selected="true">
         <Property name="p_rptTxt_2119_1" order="26" propName="reportText" val="{no text}" />
        </ListItem>
       </List>
      </ListField>
     </Question>

## Read Only Question

The question above is a special kind of read only question that is displayed to the user, but they cannot fill it in. It is informational in nature only. 

In [ ]:
 <ListItem name="LI_45594" order="192" ID="45594.100004300" title="Other adjacent organs and structures (specify)">
         <Property name="p_rptTxt_45594_1" order="193" propName="reportText" val="{no text}" />
         <ListItemResponseField name="lirf_45594_2" order="194" responseRequired="true">
          <Response name="rsp_45594_3" order="195">
           <string name="str_45594_4" order="196" />
          </Response>
         </ListItemResponseField>
        </ListItem>

## ListItem With Response

There are many questions in SDC forms which might have a response section 